A. Ik wil Anaconda gebruiken:

Installeer zo nodig eerst Anaconda. Download Anaconda van https://www.anaconda.com/download

In Anaconda navigator klikt u in het menu aan de linkerzijde van het scherm op Environments.
Maak een nieuwe virtuele omgeving aan met create.
Selecteer uw nieuwe virtuele omgeving door er eenmaal op te klikken. Klik vervolgens op het witte pijltje in het groene rondje.
Kies Open terminal. Voor de opdrachtprompt ziet u de padnaam staan waarin u het bestand requirements.txt moet copiëren dat u aantreft in de map Installatie van het workshop dossier.
Type in de terminal 'pip3 install -r requirements.txt' en druk op enter.
Wacht tot de modules zijn geïnstalleerd.

Installeer zonodig de modules van jupiter notebook door in het rechterdeel van het scherm het pakket jupyter te selecteren en installeren.

U kunt nu uw notebook gebruiken.



of



B. Ik wil Visual Studio Code gebruiken:

Installeer zonodig eerst python. Download Python versie 3.10.7 van https://www.python.org/downloads/

Open een windows terminal door in de windows zoekbalk 'cmd' te typen en op het icoontje te klikken.
Ga in de terminal naar de map 'installatie' van het workshop dossier met het commando 'cd c:/padnaardemapinstallatie'. In deze map staat het bestand 'requirements.txt'.
Type 'pip3 install -r requirements.txt' en druk op enter.
Wacht tot de modules zijn geïnstalleerd.

Installeer zo nodig eerst Visual Studio Code. Download Microsoft Visual Studio Code versie 1.85.0 van https://code.visualstudio.com/download

Start Visual Studio Code Klik in Visual Studio Code op het incoontje van 'Extensions' in het lint aan de linkerkant van het schermboven.
De beschikbare extensions worden nu in een lijst getoond. Type in de zoekbalk boven de lijst 'Jupyter' om de notebook extensie te vinden en te installeren.
Klik op de regel van de Jupiter extensie. In het rechterscherm ziet u nu de details van de extensie. Klik op 'Install' om de extensie te installeren.

U kunt nu uw notebook in Visual Studio Code gebruiken.



De volgende python bibliotheken ('modules') worden gebruikt:

In [ ]:
# run deze code eenmalig om de benodigde software bibliotheken ('modules') te installeren
%pip install chromadb==0.4.18
%pip install openai==1.3.8
%pip install langchain==0.0.348
%pip install huggingface-hub==0.16.4
%pip install pypdf==3.17.2
%pip install openpyxl==3.0.10
%pip install docx2txt==0.8
%pip install unstructured[all-docs]==0.11.2
%pip install sentence-transformers==2.2.2
%pip install streamlit==1.29.0

1. Importeer de benodigde software bibliotheken

In [ ]:
import os
from langchain.document_loaders.pdf import PyPDFLoader
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.document_loaders.word_document import UnstructuredWordDocumentLoader
from langchain.document_loaders.excel import UnstructuredExcelLoader
from langchain.document_loaders.powerpoint import UnstructuredPowerPointLoader
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.chroma import Chroma
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from huggingface_hub import hf_hub_download, login

2. Definieer de parameters

In [ ]:
CURRENT_WORKING_DIRECTORY = os.getcwd() # NB gebruik GEEN mapnamen met spaties want dat resulteert in een onjuist pad en een foutmelding!!!
DOCUMENTEN_FOLDER_NAME = os.path.join(CURRENT_WORKING_DIRECTORY, 'documenten') # NB gebruik GEEN mapnamen met spaties want dat resulteert in een onjuist pad en een foutmelding!!!
EMBEDDINGS_FOLDER_NAME = os.path.join(CURRENT_WORKING_DIRECTORY, 'embeddingmodel') # NB gebruik GEEN mapnamen met spaties want dat resulteert in een onjuist pad en een foutmelding!!!
VECTORDB_FOLDER_NAME = os.path.join(CURRENT_WORKING_DIRECTORY, 'vectordb') # NB gebruik GEEN mapnamen met spaties want dat resulteert in een onjuist pad en een foutmelding!!!
NAME_VECTOR_DB = 'workshop'
NAME_COLLECTION = 'workshop_dossier'
OPENAI_API_KEY = "GEEF HIER UW KEY IN"
HUGGINGFACE_API_KEY = "GEEF HIER UW KEY IN"

3. Lees document(en) in

In [ ]:
for file in os.listdir(DOCUMENTEN_FOLDER_NAME):
    os.rename(os.path.join(DOCUMENTEN_FOLDER_NAME, file), os.path.join(DOCUMENTEN_FOLDER_NAME, file).replace(' ', '_').lower()) # NB gebruik GEEN mapnamen met spaties want dat resulteert in een onjuist pad en een foutmelding!!!

loaders = {
        ".pdf": PyPDFLoader,
        ".csv": CSVLoader,
        ".docx": UnstructuredWordDocumentLoader,
        ".xlsx": UnstructuredExcelLoader,
        ".pptx": UnstructuredPowerPointLoader,
        ".txt": TextLoader
    }

def maak_de_directory_loader(file_type, directory_path):
    return DirectoryLoader(path=directory_path, glob=f"**/*{file_type}", loader_cls=loaders[file_type], show_progress=True, use_multithreading=False, max_concurrency=1)

pdf_loader = maak_de_directory_loader(".pdf", DOCUMENTEN_FOLDER_NAME)
csv_loader = maak_de_directory_loader(".csv", DOCUMENTEN_FOLDER_NAME)
docx_loader = maak_de_directory_loader(".docx", DOCUMENTEN_FOLDER_NAME)
xlsx_loader = maak_de_directory_loader(".xlsx", DOCUMENTEN_FOLDER_NAME)
pptx_loader = maak_de_directory_loader(".pptx", DOCUMENTEN_FOLDER_NAME)
txt_loader = maak_de_directory_loader(".txt", DOCUMENTEN_FOLDER_NAME)

documents = []

for file in os.listdir(DOCUMENTEN_FOLDER_NAME):
    print(file)
    if file.endswith('.pdf'):
        documents.extend(pdf_loader.load())
    elif file.endswith(".csv"):
        documents.extend(csv_loader.load())
    elif file.endswith(".docx"):
        documents.extend(docx_loader.load())
    elif file.endswith(".xlsx"):
        documents.extend(xlsx_loader.load())
    elif file.endswith(".pptx"):
        documents.extend(pptx_loader.load())
    elif file.endswith(".txt"):
        documents.extend(txt_loader.load())
    elif file.endswith(".doc") or file.endswith(".xls") or file.endswith(".ppt"):
        print("")
        print("Document", file, "NIET verwerkt!!!")
    else:
        print("")
        print("Document", file, "NIET verwerkt!!!")

4. Knip document in kleinere tekstfragmenten

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=750, chunk_overlap=75, separators=["\n\n", "\n", " ", ""], length_function=len,) # 256 tokens max; bij 3 karakters per token dus ongeveer 750 karakters
chunked_documents = text_splitter.split_documents(documents)
print(len(chunked_documents))

5. Definieer het LLM (large language model) voor de embedding van de tekstfragmenten

In [ ]:
emb_model = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=emb_model, cache_folder=EMBEDDINGS_FOLDER_NAME)
login(token = HUGGINGFACE_API_KEY)

#text = 'Dit is een voorbeeld'
#embedding = embeddings.embed_query(text)
#print(len(embedding))
#print(embedding[:5])

6. Tokenize en embedd de tekstfragmenten en sla de word embeddings op in vector store

In [ ]:
vector_db_path = VECTORDB_FOLDER_NAME
vector_db = Chroma.from_documents(documents=chunked_documents, embedding=embeddings, collection_name=NAME_COLLECTION, persist_directory=vector_db_path)
vector_db.persist()

7. Definieer het LLM (large language model) voor de chat interface

In [ ]:
llm = ChatOpenAI(model='gpt-3.5-turbo', openai_api_key=OPENAI_API_KEY, temperature=0.3, max_tokens=2000)

8. Maak de vraag-antwoord pipeline

In [ ]:
emb_model = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=emb_model, cache_folder=EMBEDDINGS_FOLDER_NAME)
vector_db_name = NAME_VECTOR_DB 
vector_db_folder = VECTORDB_FOLDER_NAME
persist_dir = os.path.join(vector_db_folder, vector_db_name)
chain = load_qa_chain(llm, chain_type="stuff")

9. Stel een vraag aan de chatbot: zoek in de vector db tekstfragmenten voor de context, stop deze context samen met de vraag in een prompt (query) en verkrijg een antwoord via de llm voor de tekst interface

In [ ]:
q = "Wat is warme grond?"
v = vector_db.similarity_search(q, k=4, include_metadata=True)
res = chain({"input_documents": v, "question": q})
print(res["output_text"])